In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE163114"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE163114"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE163114.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE163114.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE163114.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Ki-67 promotes carcinogenesis by enabling global transcriptional programmes"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: HeLa'], 1: ['lentivirus: shRNA control', 'lentivirus: shRNA Ki-67']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The dataset appears to be from a HeLa cell line with shRNA experiments
# As a SuperSeries with SubSeries, it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: Looking at row 1, it shows two different treatments (shRNA control vs shRNA Ki-67)
# This is the experimental condition and should be our trait of interest
trait_row = 1

# Age and gender: Not available in this cell line dataset
age_row = None
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """
    Convert shRNA treatment information to a binary indicator.
    shRNA control = 0, shRNA Ki-67 = 1
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'control' in value:
        return 0
    elif 'ki-67' in value:
        return 1
    else:
        return None

def convert_age(value):
    # Not applicable for this dataset
    return None

def convert_gender(value):
    # Not applicable for this dataset
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # For GEO data, we need to load the clinical data file that was created in a previous step
        # Attempt to find the clinical data file in the expected location
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
            
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age,
                gender_row=gender_row,
                convert_gender=convert_gender
            )
            
            # Preview the dataframe
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Ensure the output directory exists
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save to CSV
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print(f"Clinical data file not found at {clinical_data_path}")
            print("This dataset may require manual inspection.")
    except Exception as e:
        print(f"Error processing clinical data: {e}")
        print("This dataset may require special handling.")


Clinical data file not found at ../../input/GEO/Cervical_Cancer/GSE163114/clinical_data.csv
This dataset may require manual inspection.


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The index values ['1', '2', '3', '4', ...] appear to be simple numeric identifiers,
# not standard human gene symbols which typically follow formats like "TP53", "BRCA1", etc.
# These are likely probe IDs or some other platform-specific identifiers that need to be
# mapped to proper gene symbols for biological interpretation.

# Based on my biomedical knowledge, these are not human gene symbols and require mapping

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the output, we need to identify which columns to use for gene mapping
# Looking at the gene annotation preview, we can see:
# - 'ID' contains the numeric identifiers matching the gene expression data index
# - 'GENE_SYMBOL' contains the actual gene symbols (e.g., HEBP1, KCNE4)

# 2. Extract the gene mapping dataframe from gene annotation
gene_mapping = get_gene_mapping(
    annotation=gene_annotation,
    prob_col='ID',  # Gene identifiers column
    gene_col='GENE_SYMBOL'  # Gene symbols column
)

# 3. Convert probe-level measurements to gene expression data by applying gene mapping
gene_data = apply_gene_mapping(
    expression_df=gene_data,
    mapping_df=gene_mapping
)

# Print first few rows of the gene data after mapping to verify
print("Gene expression data after mapping (first 5 rows):")
print(gene_data.head(5))


Gene expression data after mapping (first 5 rows):
          GSM4950455  GSM4950456  GSM4950457  GSM4950458  GSM4950459  \
Gene                                                                   
A1BG      132.083288  129.833098   95.850230  100.615695  124.773355   
A1BG-AS1   16.155110   14.861680    4.716991   11.106840    8.644419   
A1CF      212.139998  173.847205   54.092575  156.444192  106.723514   
A2LD1     244.871530  180.816300  109.013836  173.861860  247.707170   
A2M       133.594765  128.883865   62.684670  123.532323   81.277947   

          GSM4950460  GSM4950461  GSM4950462  GSM4950463  GSM4950464  \
Gene                                                                   
A1BG      109.751668  108.931347   92.967957  127.500770  106.803252   
A1BG-AS1    4.350618   10.533510    7.061245   13.553190   10.343080   
A1CF       76.701249  157.344856  122.121485  154.139703  196.274230   
A2LD1     145.602781  206.909400  220.527440  229.859650  200.153210   
A2M        5

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Extract clinical features directly from the matrix file data (reuse the data from previous steps)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Use the correct trait conversion function from Step 2
def convert_trait(value):
    """
    Convert shRNA treatment information to a binary indicator.
    shRNA control = 0, shRNA Ki-67 = 1
    """
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value = value.lower()
    if 'control' in value:
        return 0
    elif 'ki-67' in value:
        return 1
    else:
        return None

# Extract clinical features using the trait_row identified in Step 2
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=1,  # Using row 1 which contains shRNA treatment information
    convert_trait=convert_trait
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Now link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print("Linked data shape:", linked_data.shape)

# Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="HeLa cell line samples treated with shRNA control vs. shRNA Ki-67 knockdown."
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE163114.csv
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE163114.csv
Linked data shape: (12, 19848)


For the feature 'Cervical_Cancer', the least common label is '0.0' with 6 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Cervical_Cancer' in this dataset is fine.



Linked data saved to ../../output/preprocess/Cervical_Cancer/GSE163114.csv
